### Importing Necessary Libraries

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_community.chat_models import ChatOllama

_ = load_dotenv()

### Setting Environment Variables

In [2]:
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")

### Initializing GraphCypherQAChain

In [3]:
graph = Neo4jGraph()
cypher_llm = ChatOpenAI(model="gpt-4-turbo")
qa_llm = ChatOpenAI(model="gpt-4-turbo")
chain = GraphCypherQAChain.from_llm(graph=graph, llm=cypher_llm, verbose=True, validate_cypher=True)

### Example Queries

In [4]:
response = chain.invoke({"query": "Which games have same publisher in the year 2003?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g1:Game)-[:PUBLISHED_BY]->(p:Publisher)<-[:PUBLISHED_BY]-(g2:Game)
WHERE g1.year = 2003 AND g2.year = 2003 AND g1.name <> g2.name
RETURN g1.name AS Game1, g2.name AS Game2, p.name AS Publisher

Full Context:
[{'Game1': 'Super Mario Bros. 3', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Pokémon Colosseum', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario & Luigi: Superstar Saga', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario Party 5', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Donkey Kong Country', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Kirby Air Ride', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Pokemon Pinball: Ruby & Sapphire', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario Golf: To

{'query': 'Which games have same publisher in the year 2003?',
 'result': 'The games that have the same publisher in the year 2003 are Super Mario Bros. 3, Pokémon Colosseum, Mario & Luigi: Superstar Saga, Mario Party 5, Donkey Kong Country, Kirby Air Ride, Pokemon Pinball: Ruby & Sapphire, Mario Golf: Toadstool Tour, Final Fantasy: Crystal Chronicles, and WarioWare, Inc.: Mega MicroGame$, all published by Nintendo.'}

In [5]:
print(response['result'])

The games that have the same publisher in the year 2003 are Super Mario Bros. 3, Pokémon Colosseum, Mario & Luigi: Superstar Saga, Mario Party 5, Donkey Kong Country, Kirby Air Ride, Pokemon Pinball: Ruby & Sapphire, Mario Golf: Toadstool Tour, Final Fantasy: Crystal Chronicles, and WarioWare, Inc.: Mega MicroGame$, all published by Nintendo.


In [6]:
response = chain.invoke({"query": "Find pairs of games published by the same publisher in the year 2003 and their total combined global sales."})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g1:Game)-[:PUBLISHED_BY]->(p:Publisher)<-[:PUBLISHED_BY]-(g2:Game)
WHERE g1.year = 2003 AND g2.year = 2003 AND g1 <> g2
RETURN g1.name AS Game1, g2.name AS Game2, p.name AS Publisher, g1.global_sales + g2.global_sales AS TotalGlobalSales

Full Context:
[{'Game1': 'Super Mario Bros. 3', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo', 'TotalGlobalSales': 12.15}, {'Game1': 'Pokémon Colosseum', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo', 'TotalGlobalSales': 9.49}, {'Game1': 'Mario & Luigi: Superstar Saga', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo', 'TotalGlobalSales': 9.120000000000001}, {'Game1': 'Mario Party 5', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo', 'TotalGlobalSales': 9.030000000000001}, {'Game1': 'Donkey Kong Country', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo', 'TotalGlobalSales': 8.99}, {'Game1': 'Kirby Air Ride

{'query': 'Find pairs of games published by the same publisher in the year 2003 and their total combined global sales.',
 'result': 'The pairs of games published by Nintendo in 2003 along with their total combined global sales are as follows:\n- "Super Mario Bros. 3" and "Mario Kart: Double Dash!!" with a total of 12.15 million in global sales.\n- "Pokémon Colosseum" and "Mario Kart: Double Dash!!" with a total of 9.49 million in global sales.\n- "Mario & Luigi: Superstar Saga" and "Mario Kart: Double Dash!!" with a total of 9.12 million in global sales.\n- "Mario Party 5" and "Mario Kart: Double Dash!!" with a total of 9.03 million in global sales.\n- "Donkey Kong Country" and "Mario Kart: Double Dash!!" with a total of 8.99 million in global sales.\n- "Kirby Air Ride" and "Mario Kart: Double Dash!!" with a total of 8.57 million in global sales.\n- "Pokemon Pinball: Ruby & Sapphire" and "Mario Kart: Double Dash!!" with a total of 8.51 million in global sales.\n- "Mario Golf: Toadstool

In [7]:
print(response['result'])

The pairs of games published by Nintendo in 2003 along with their total combined global sales are as follows:
- "Super Mario Bros. 3" and "Mario Kart: Double Dash!!" with a total of 12.15 million in global sales.
- "Pokémon Colosseum" and "Mario Kart: Double Dash!!" with a total of 9.49 million in global sales.
- "Mario & Luigi: Superstar Saga" and "Mario Kart: Double Dash!!" with a total of 9.12 million in global sales.
- "Mario Party 5" and "Mario Kart: Double Dash!!" with a total of 9.03 million in global sales.
- "Donkey Kong Country" and "Mario Kart: Double Dash!!" with a total of 8.99 million in global sales.
- "Kirby Air Ride" and "Mario Kart: Double Dash!!" with a total of 8.57 million in global sales.
- "Pokemon Pinball: Ruby & Sapphire" and "Mario Kart: Double Dash!!" with a total of 8.51 million in global sales.
- "Mario Golf: Toadstool Tour" and "Mario Kart: Double Dash!!" with a total of 8.48 million in global sales.
- "Final Fantasy: Crystal Chronicles" and "Mario Kart: D

In [11]:
response = chain.invoke({"query": "Find the most successful publisher in 2008 in terms of global sales and list their top three games"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:Game)-[:PUBLISHED_BY]->(p:Publisher)
WHERE g.year = 2008
RETURN p.name AS Publisher, SUM(g.global_sales) AS TotalGlobalSales, COLLECT(g.name)[0..3] AS TopThreeGames
ORDER BY TotalGlobalSales DESC
LIMIT 1;

Full Context:
[{'Publisher': 'Nintendo', 'TotalGlobalSales': 91.22000000000004, 'TopThreeGames': ['Mario Kart Wii', 'Super Smash Bros. Brawl', 'Pokémon Platinum Version']}]

> Finished chain.


{'query': 'Find the most successful publisher in 2008 in terms of global sales and list their top three games',
 'result': 'Nintendo was the most successful publisher in 2008 in terms of global sales. Their top three games were Mario Kart Wii, Super Smash Bros. Brawl, and Pokémon Platinum Version.'}

In [12]:
print(response['result'])

Nintendo was the most successful publisher in 2008 in terms of global sales. Their top three games were Mario Kart Wii, Super Smash Bros. Brawl, and Pokémon Platinum Version.


In [13]:
response = chain.invoke({"query": "Determine the average sales of games by publishers who released at least two games in 2001 and compare these averages."})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:Game)-[:PUBLISHED_BY]->(p:Publisher)
WHERE g.year = 2001
WITH p, COUNT(g) AS games_count
WHERE games_count >= 2
MATCH (g2:Game)-[:PUBLISHED_BY]->(p)
RETURN p.name AS Publisher, AVG(g2.global_sales) AS Average_Global_Sales
ORDER BY Average_Global_Sales DESC

Full Context:
[{'Publisher': 'Nintendo', 'Average_Global_Sales': 2.5096866096866064}, {'Publisher': 'Microsoft Game Studios', 'Average_Global_Sales': 1.3004761904761897}, {'Publisher': 'Enix Corporation', 'Average_Global_Sales': 1.1246666666666663}, {'Publisher': 'LucasArts', 'Average_Global_Sales': 0.9704444444444442}, {'Publisher': 'Take-Two Interactive', 'Average_Global_Sales': 0.9674092009685233}, {'Publisher': 'Sony Computer Entertainment', 'Average_Global_Sales': 0.889458272327965}, {'Publisher': 'Electronic Arts', 'Average_Global_Sales': 0.8218504811250932}, {'Publisher': 'Universal Interactive', 'Average_Global_Sales': 0.7726086956521738}, {'Publi

{'query': 'Determine the average sales of games by publishers who released at least two games in 2001 and compare these averages.',
 'result': "Nintendo leads with an average of approximately 2.51 million global sales per game. Microsoft Game Studios follows with about 1.30 million, while Enix Corporation holds an average of 1.12 million. LucasArts and Take-Two Interactive are close, with averages of 0.97 million and 0.97 million respectively. Sony Computer Entertainment's average stands at 0.89 million, Electronic Arts at 0.82 million, Universal Interactive at 0.77 million, Activision at 0.75 million, and Virgin Interactive rounds out the list with an average of 0.71 million global sales per game."}

In [14]:
print(response['result'])

Nintendo leads with an average of approximately 2.51 million global sales per game. Microsoft Game Studios follows with about 1.30 million, while Enix Corporation holds an average of 1.12 million. LucasArts and Take-Two Interactive are close, with averages of 0.97 million and 0.97 million respectively. Sony Computer Entertainment's average stands at 0.89 million, Electronic Arts at 0.82 million, Universal Interactive at 0.77 million, Activision at 0.75 million, and Virgin Interactive rounds out the list with an average of 0.71 million global sales per game.


In [15]:
response = chain.invoke({"query": "Identify publishers who released games in both 2001 and 2002, and compare their total global sales for each year"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g2001:Game)-[:PUBLISHED_BY]->(p:Publisher)
WHERE g2001.year = 2001
WITH p, SUM(g2001.global_sales) AS sales2001
MATCH (g2002:Game)-[:PUBLISHED_BY]->(p)
WHERE g2002.year = 2002
RETURN p.name AS Publisher, sales2001, SUM(g2002.global_sales) AS sales2002

Full Context:
[{'Publisher': 'Nintendo', 'sales2001': 45.37000000000002, 'sales2002': 48.30999999999999}, {'Publisher': 'Microsoft Game Studios', 'sales2001': 13.490000000000002, 'sales2002': 6.959999999999999}, {'Publisher': 'Take-Two Interactive', 'sales2001': 20.429999999999996, 'sales2002': 20.29}, {'Publisher': 'Sony Computer Entertainment', 'sales2001': 43.29, 'sales2002': 25.850000000000012}, {'Publisher': 'Activision', 'sales2001': 18.590000000000003, 'sales2002': 25.939999999999994}, {'Publisher': 'Ubisoft', 'sales2001': 4.99, 'sales2002': 16.21}, {'Publisher': 'Electronic Arts', 'sales2001': 45.12000000000001, 'sales2002': 73.01000000000002}, {'Publish

{'query': 'Identify publishers who released games in both 2001 and 2002, and compare their total global sales for each year',
 'result': "In 2001 and 2002, several publishers released games and had varying sales figures. Nintendo had sales of 45.37 million in 2001 and 48.31 million in 2002. Microsoft Game Studios reported sales of 13.49 million in 2001 and 6.96 million in 2002. Take-Two Interactive had nearly consistent sales with 20.43 million in 2001 and 20.29 million in 2002. Sony Computer Entertainment saw a decrease from 43.29 million in 2001 to 25.85 million in 2002. Activision's sales rose from 18.59 million in 2001 to 25.94 million in 2002. Ubisoft significantly increased their sales from 4.99 million in 2001 to 16.21 million in 2002. Electronic Arts had a notable increase in sales from 45.12 million in 2001 to 73.01 million in 2002. Sega's sales also increased from 6.6 million in 2001 to 12.3 million in 2002. SquareSoft had a decrease in sales, with 0.25 million in 2001 and 0.

In [16]:
print(response['result'])

In 2001 and 2002, several publishers released games and had varying sales figures. Nintendo had sales of 45.37 million in 2001 and 48.31 million in 2002. Microsoft Game Studios reported sales of 13.49 million in 2001 and 6.96 million in 2002. Take-Two Interactive had nearly consistent sales with 20.43 million in 2001 and 20.29 million in 2002. Sony Computer Entertainment saw a decrease from 43.29 million in 2001 to 25.85 million in 2002. Activision's sales rose from 18.59 million in 2001 to 25.94 million in 2002. Ubisoft significantly increased their sales from 4.99 million in 2001 to 16.21 million in 2002. Electronic Arts had a notable increase in sales from 45.12 million in 2001 to 73.01 million in 2002. Sega's sales also increased from 6.6 million in 2001 to 12.3 million in 2002. SquareSoft had a decrease in sales, with 0.25 million in 2001 and 0.12 million in 2002. Atari saw a significant increase from 8.05 million in 2001 to 21.25 million in 2002. These figures reflect how each pu

### Reference
- video game sales dataset: https://www.kaggle.com/datasets/gregorut/videogamesales
- Neo4j + Langchain: https://python.langchain.com/v0.2/docs/integrations/graphs/neo4j_cypher/